# Change-Point Detection in Business Cycles

In [1]:
import pandas as pd
import numpy as np
import matplotlib


In [2]:
# These are exchange rate data's, indexed, not actual values
neer_data = pd.read_excel('../data/processed_data/georgia_neer_data_cleaned_processed.xlsx')
reer_data = pd.read_excel('../data/processed_data/georgia_reer_data_cleaned_processed.xlsx')

# This is GDP data

gdp_data = pd.read_excel('../data/processed_data/georgia_monthly_gdp_chow_lin.xlsx')


#We must add neer and reer to the gdp data, by date.

# Merge NEER and REER data with GDP data
gdp_data = gdp_data.merge(neer_data, on='Date', how='left')
gdp_data = gdp_data.merge(reer_data, on='Date', how='left')

ValueError: You are trying to merge on datetime64[ns] and object columns for key 'Date'. If you wish to proceed you should use pd.concat

In [3]:
neer_data

,Date,NEER (Dec-2009=100)
0,01-2010,98.270566
1,02-2010,99.272945
2,03-2010,99.567377
3,04-2010,97.667794
4,05-2010,98.616040
...,...,...
179,12-2024,180.937478
180,01-2025,179.925379
181,02-2025,178.915148
182,03-2025,178.482231
